In [1]:
#!pip install transformers
#!pip install datasets


In [2]:
#!pip install seqeval

# Skript2: Model Finetuning mit Hyperparametersuche für Text-Klassifizierung mit germaneval2019 Task2 -Subtask3

Dieses Notebook beschreibt das Training eines Text-Klassifizierungsmodells für den Datensatz GermEval 2019 Task2 -Subtask3.
Es wird ein BERT - Modell mit der nativen Hugging Face Transformer Bibliothek trainiert. Dafür wird vor dem Training mithilfe automatischer Hyperparametersuche versucht, bessere Trainingsparameter zu finden, um die Modellleistung zu verbessern. Das Training erfolgt nach dem gleichen Split wie es der Shared Task 2019 vorgab.

In [3]:
# Importieren der Hugging Face Datesets Bibliothek
import datasets

In [4]:
# Parameter für die Scriptsteuerung:
# die Task für die das Modell trainiert werden soll
task = "task3"
# das zu verwendende pretrained Transformer Modell
model_checkpoint = "deepset/gbert-base"
# die Trainingsbatchsize
batch_size = 4

## Preprocessing des Datasets
Zunächst müssen die Daten aus dem GermEval2019 Datensatz vorverarbeitet werden. Dazu werden diese in einen `Pandas DataFrames` umgewandelt.

In [5]:
import pandas as pd

Als ersten werden mit Hilfe der `read_csv()` Methode die Daten für das Trainings- und das Valiederungsset zu einem `DataFrame` zusammengefügt. Dabei werden die Werte in den Labelspalten in numerische Kategorienencodings umgewandelt.

In [6]:
data_all=pd.read_csv("../datasets/germeval2019/train_valid/germeval2019.training_subtask3.txt",delimiter="\t", header=None,quoting=3)
data_all.columns = ["text","task1_label","task2_label","task3_label"]
# Umwandlung der Label in die jeweiligen numerischen Encodings
data_all.task1_label = data_all.task1_label.astype('category').cat.codes
data_all.task2_label = data_all.task2_label.astype('category').cat.codes
data_all.task3_label = data_all.task3_label.astype('category').cat.codes
data_all

,text,task1_label,task2_label,task3_label
0,@spdde kein verläßlicher Verhandlungspartner. ...,0,1,0
1,@milenahanm 33 bis 45 habe ich noch gar nicht ...,0,2,0
2,@tagesschau Euere AfD Hetze wirkt. Da könnt ih...,0,0,0
3,"Deutsche Medien, Halbwahrheiten und einseitige...",0,0,0
4,@Ralf_Stegner Oman Ralle..dich mag ja immer no...,0,1,0
...,...,...,...,...
1953,@Alltags_Kotze Dein Feminismus und Genderquats...,0,0,0
1954,@UdoUlfkotte Hauptsache den Asylanten gehts ge...,0,0,1
1955,@SteinbachErika Ich finde AFD Wähler besser al...,0,1,0
1956,"@RKnillmann @lawyerberlin @AfD Aha, der Islam ...",0,0,1


Für das Testdataset wird ebenfalls ein `DataFrame` erzeugt und die Label encodiert.

In [7]:
data_test = pd.read_csv("../datasets/germeval2019/test/germeval2019GoldLabelsSubtask3.txt",delimiter="\t", header=None, quoting=3)
data_test.columns = ["text","task1_label","task2_label","task3_label"]
data_test.task1_label = data_test.task1_label.astype('category').cat.codes
data_test.task2_label = data_test.task2_label.astype('category').cat.codes
data_test.task3_label = data_test.task3_label.astype('category').cat.codes
data_test

,text,task1_label,task2_label,task3_label
0,"Für mich ist der morgige Tag ein Tag der ""Größ...",0,0,0
1,"@ndaktuell Ach Gott mehr nicht,was gedenkt man...",0,0,0
2,@gerdm4863 Das ist einfach so...aber noch lang...,0,0,0
3,@welt Bla bla bla! Lügenpresse verkauft uns ma...,0,0,0
4,@OnlineMagazin Das kann doch nur ein Fake sein...,0,0,0
...,...,...,...,...
925,"@Schminkflinte @SawsanChebli Mir doch egal, ob...",0,0,0
926,@rspctfl @ThomasMichael71 @Innenwelttramp @erg...,0,0,0
927,Merkel zerstört den bürgerlich-konservativen K...,0,0,0
928,Feminismus ist der Islamismus der Weiblichkeit.,0,0,0


Anschließend kann aus den `Pandas DataFrames` ein `Hugging Face Dataset` generiert werden. Hierfür wird zunächst ein `Feature` Dictionary erzeugt. Dieses enthält Typisierungen für die Features des Datasets. So werden die Features `task1_label` und `task2_label` jeweils als `ClassLabel` typisiert und das `text` Feature wird als String-Value deklariert.

In [8]:
# Erzeugen eines DataSet Objektes mit allen Daten
features = datasets.Features({
  #'__index_level_0__': datasets.Value(dtype='int32', id=None),
  'task3_label': datasets.ClassLabel(num_classes=2, names=['EXPLICIT', 'IMPLICIT'], names_file=None, id=None),
  'text': datasets.Value(dtype='string', id=None)
})
dataset = datasets.Dataset.from_pandas(data_all,features=features)
dataset_test = datasets.Dataset.from_pandas(data_test,features=features)

Beispielausgabe der Dataset Objekte:

In [9]:
dataset_test, dataset

(Dataset({
     features: ['task3_label', 'text'],
     num_rows: 930
 }),
 Dataset({
     features: ['task3_label', 'text'],
     num_rows: 1958
 }))

#### Split des Datasets
Mithilfe der der Dataset Objekte wird anschließend der Trainings- und Validierungssplit durchgeführt. Die gewählte größe des Validierungsset beträgt 20% des Datasets:

In [10]:
# Aufsplitten des Trainingssets in 80/20 (Training/Validation)Split
dataset_train_valid = dataset.train_test_split(test_size=0.2, seed=42)

Die `train_test_split()` Methode gibt ein `DatasetDict` Objekt zurück. Dies ist ein Dictionary mit einem Trainings- und einem Testdataset. Das Testdataset stellt in diesem Fall das Validierungsdataset dar.

In [11]:
dataset_train_valid

DatasetDict({
    train: Dataset({
        features: ['task3_label', 'text'],
        num_rows: 1566
    })
    test: Dataset({
        features: ['task3_label', 'text'],
        num_rows: 392
    })
})

Um ein gemeinsames Dataset Objekt zu erhalten, werden die Datasets anschließend zusammengefügt und entsprechend des Splits benannt:

In [12]:
# Zusammenfügen aller DataSet-Objekte zu einem gemeinsamen DataSet mit benannten Splits
dataset_train_valid_test = datasets.DatasetDict({"train": dataset_train_valid["train"],
                                                 "valid": dataset_train_valid["test"],
                                                 "test": dataset_test})

Aufbau des gesamten Dataset Dictionary:

In [13]:
dataset_train_valid_test

DatasetDict({
    train: Dataset({
        features: ['task3_label', 'text'],
        num_rows: 1566
    })
    valid: Dataset({
        features: ['task3_label', 'text'],
        num_rows: 392
    })
    test: Dataset({
        features: ['task3_label', 'text'],
        num_rows: 930
    })
})

Funktion für die Ausgabe von Beispieldaten aus dem Dataset:

In [14]:
# Quelle in Anlehnung an: https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb#scrollTo=X6HrpprwIrIz
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10,seed=None):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    random.seed(seed)
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            random.seed(seed)
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

Ausgabe von Beispieldaten aus dem Trainingsdataset:

In [15]:
show_random_elements(dataset_train_valid_test["train"],seed=42)

,task3_label,text
0,EXPLICIT,@MDRINFO @KontraKulturell |LBR| ab nach Hause mit dem Pack.
1,EXPLICIT,@Heinrich_Krug @krippmarie Sorry aber das ist schon gruselig wie Merkel aussieht....irgendwie passt die optisch besser zu den Afghanen oder Syrern 😡😡
2,EXPLICIT,"Warum hat wohl der Terrorfürst Yassir #Arafat das Wort »#Palästinenser« für die #Araber in der #Levante benutzt? Dieses Volk gab es vorher nicht, aber man bediente sich aus dem #Antisemitismus der #Römer, die uns #Juden demütigten &amp; Eretz #Israel in #Palästina« unbenannten."
3,IMPLICIT,@BonkeKolja Was hätte A. H. mit solche skupellosen Tätern gemacht? Sicher das Richtige!
4,EXPLICIT,"@DoraBromberger Hoffentlich bekommt die dumme, gewaltbereite Antifantenbrut richtig auf die Fresse! ☻👹🤘 |LBR| Respekt für friedliche Demonstranten! 👍 👍"
5,EXPLICIT,@SchauHin2 |LBR| Hoffentlich sind die Nationalisten dann die Sieger nicht die Asylantenbeschützer und Asyalantenabschaum?
6,EXPLICIT,@stefanolix @teite99 Nutzbringende Migration bedingt Abwehr der nutzlosen Migration.
7,EXPLICIT,@sentenza66 DIESES GESAMTE GESOCKSE MIT IHREM CLAN MUSS INNERHALB EINES TAGES RAUS AUS DEUTSCHLAND!
8,EXPLICIT,"@tagesschau Warum schreibt ihr Merkel Vasallen nicht mal die Wahrheit über Merkels Verbrech? Ach ja, dann seid ihr ja Arbeitslos!!!"
9,EXPLICIT,@ArasBacho @dushanwegner Eine dumme Labertasche dieser @ArasBacho ...


# Preprocessing der Daten

Beschaffen des zugehörigen Tokenizers zum ausgewählten Modell:

In [16]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Der Tokenizer kann direkt genutzt werden, um eine Inputsequenz zu tokenisieren. Dabei erhält man ein Dictionary mit Mappings zu den input_ids, token_type_ids und eine attention_mask. Die input_id ist die Identifikation des jeweiligen Tokens im Vokabluar des Modells. Token_type_ids markieren Tokens in Seq2Seq Tasks und geben dem Modell Informationen darüber, zu welchem Teil einer zweiteiligen Eingabesequenz ein Token gehört. Die attention_mask teilt dem Modell mit, für welche Token die Attention berechnet werden soll. Ist eine Eingabesequenz z. B. sehr kurz im Gegensatz zu den anderen, dann wird diese per Padding auf die gleiche Länge gebracht. Die attention_mask verhindert anschließend, dass die Attention für diese Padding Token berechnet wird.

In [17]:
tokenizer("Es ist natürlich viel einfacher auch weiterhin jeden Widerstand zu skandalisieren als den Konzernen die Stirn zu bieten. Ist einfach gemütlicher, als die eigenen Fehler zu reflektieren.")

{'input_ids': [102, 479, 215, 3392, 827, 12822, 313, 3750, 2809, 8550, 205, 11473, 11545, 7232, 276, 190, 8448, 106, 128, 21429, 205, 3915, 566, 2302, 1523, 28283, 105, 818, 276, 128, 2233, 5099, 205, 21713, 11044, 566, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Um die Daten der Datasets vorzuverarbeiten wird die folgende Methode definiert\
In der Methode werden zunächst die Werte des `text` Features mit dem Tokenizer des Modells tokeniesiert und anschließend ein Label Attribut mit dem Wert der aktuellen Task eingefügt. Die Methode gibt Ausgabe des Tokenizers anschließend zurück.

In [18]:
def preprocess_data(data):
  tokenized_inputs = tokenizer(data['text'], truncation=True)
  label = data["task3_label"]
  tokenized_inputs["label"] = label
  return tokenized_inputs

Die Funktion kann beliebig viele Datensätze verarbeiten. Werden mehrere übergeben dann gibt der Tokenizer eine Liste zurück:

In [19]:
preprocess_data(dataset_train_valid_test["train"][:1])

{'input_ids': [[102, 17329, 9036, 30901, 1926, 2420, 30887, 17329, 2942, 13589, 30893, 2675, 11159, 105, 764, 778, 847, 292, 5353, 237, 566, 566, 566, 566, 103]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'label': [0]}

Anwendung der `preprocess_data` Funktion auf alle Teildatensätze im DataSet-Objekt `dataset_train_valid_test` mithilfe von `map`
 - `batched=True` beschleunigt die Verarbeitung des FastTokenizers

In [20]:
dataset_preprocessed = dataset_train_valid_test.map(preprocess_data,batched=True)

Durch das Preprocessing werden die Ausgaben des Tokenizers als Features des DataSets ergänzt.\
In der folgenden Ausgaben sieht man, dass nun die attention_mask, die input_ids, die label und die token_type_ids als Features ergänzt wurden.

In [21]:
dataset_preprocessed["train"]

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'task3_label', 'text', 'token_type_ids'],
    num_rows: 1566
})

In [22]:
show_random_elements(dataset_preprocessed["train"],seed=42,num_examples=2)

,attention_mask,input_ids,label,task3_label,text,token_type_ids
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[102, 17329, 25704, 30420, 30931, 22646, 17329, 11653, 16966, 630, 18406, 183, 14822, 18406, 404, 333, 3873, 212, 249, 13910, 566, 103]",0,EXPLICIT,@MDRINFO @KontraKulturell |LBR| ab nach Hause mit dem Pack.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[102, 17329, 4669, 2032, 20611, 30894, 17329, 15442, 2355, 1303, 110, 9147, 2690, 494, 199, 215, 778, 872, 22664, 214, 335, 6488, 18138, 566, 566, 566, 566, 11896, 10576, 128, 29872, 1903, 205, 190, 14016, 106, 394, 7791, 310, 101, 103]",0,EXPLICIT,@Heinrich_Krug @krippmarie Sorry aber das ist schon gruselig wie Merkel aussieht....irgendwie passt die optisch besser zu den Afghanen oder Syrern 😡😡,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


# Finetuning des Modells mit Hyperparametersearch

Für die Konstruktion des Modells wird ein Label-ID Mapping erzeugt. Dieses Mapping wird anschließend in der Configuration des Modells hinterlegt. Es ermöglicht dem Modell die vorhergesagten Label als Texte auszugeben anstatt ihrer numerischen Kodierung.

In [23]:
id2label = {"0":"EXPLICIT", "1":"IMPLICIT"}
label2id = {"EXPLICIT": "0", "IMPLICIT":"1"}

Erzeugen und Konfiguration des Modells mit Label-ID Mapping:
 - `AutoModelForSequenceClassification.from_pretrained` lädt automatisch das entsprechende Modell herunter und initialisiert einen Klassifizierungskopf am Ende des Modells.
 - Die auftretenden Warnungen geben nur Auskunft darüber, dass der Kopf des Modells ausgetauscht wurde und demzufolge keine trainierten Weights hat

Mit der Annahme, dass bessere gewählte Hyperparameter leistungsstärkeren Modellen führen, wird im folgenden Abschnitt mithilfe von automatischer Hyperparametersuche versucht, die Modellperformance weiter zu steigern. \
Hierfür wird das Modul optuna genutzt:

In [24]:
#!pip install optuna

Es muss eine Funktion erstellt werden, die das zu trainierende Modell nach jedem Versuch wieder initalisiert:

In [25]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 2

In [26]:
# Initialisiert ein neues Modell auf Basis des gewählten Modell Checkpoints
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id)

#### Vorbereitung für die Erzeugung des `Trainers`:
- Es müssen zunächst die `TrainingArguments` konfiguriert werden, dabei handelt es sich um die Hyperparameter des Trainings:
 - Dazu zählen z.B. die Anzahl der Epoch, die Learning Rate, die Batchsize und der weight_decay
 - `load_best_model_at_end` und `metric_for_best_model` sorgen dafür, dass am Ende des Trainings das Model mit der höchsten `F1_Score` geladen wird

In [27]:
metric_name = 'eval_f1'

args = TrainingArguments( 
    output_dir= f"{task}_transformer_hyperparametersearch",
    evaluation_strategy = "epoch",
    # empfohlener Standardwert für das Finetuning
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    # empfohlener Standardwert für das Finetuning
    weight_decay=0.01,
    load_best_model_at_end=True,
    # fp16 transformiert das Modell nach float16 um Memory zu sparen und die Trainingszeit zu verringern
    fp16=True,
    metric_for_best_model=metric_name,
)

Damit das Modell während des Trainings die gewünschten Metriken berechnen kann, muss eine Funktion definiert werden die diese Metriken berechnet. Das übernimmt die `compute_metric` Funktion:
- Die Datasets Bibliothek ermöglicht es, Funktionen zur Berechnung von Metriken herunterzuladen.

In [28]:
metric_acc = datasets.load_metric("accuracy")
metric_f1 = datasets.load_metric("f1")
metric_prec = datasets.load_metric("precision")
metric_rec = datasets.load_metric("recall")

Diese Metriken werden nun wie folgt in der `compute_metrics` Funktion berechnet. Die Funktion liefert anschließend ein Dictionary mit den vier Metriken `accuracy, precision, recall, f1` zurück.

In [29]:
import numpy as np
def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=1)
  accurracy = metric_acc.compute(predictions=predictions, references=labels)
  f1 = metric_f1.compute(predictions=predictions, references=labels, average="macro" )
  prec = metric_prec.compute(predictions=predictions, references=labels,average="macro")
  recall = metric_rec.compute(predictions=predictions, references=labels,average="macro")
 
  return {
      "accurracy" : accurracy["accuracy"],
      "precision" : prec["precision"],
      "recall": recall["recall"],
      "f1": f1["f1"]
  }

Anschließend muss ein Trainer initialisiert werden, welcher für die Hyperparametersuche genutzt wird.

In [30]:
trainer_hyper = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=dataset_preprocessed["train"].shard(index=1, num_shards=10),
    eval_dataset=dataset_preprocessed["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Festlegen der Hyperparameter die getestet werden sollen:
- In diesem Beispiel werden Werte für die Learning Rate und die Anzahl der Epochs ermittelt. 
- Die Batchsize wurde auf 4 gesetzt um Out of Memoryprobleme zu vermeiden

In [31]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
        #Festlegen er maximalen Batchsize um Cuda out of Memory zu vermeiden
        #"per_device_train_batch_size": trial.suggest_int("per_device_train_batch_size", 4,4),
        #"per_device_eval_batch_size": trial.suggest_int("per_device_eval_batch_size",4,4),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }

Für die Hyperparametersuche muss festgelegt werden, wie ein Durchlauf bewertetet werden soll.
Hierfür werden mit der folgenden Funktion alle vom Modell berechneten Metriken aufsummiert. Die Summe ist der Score für die Epoch des jeweiligen Versuchs:

In [32]:
import copy
def my_objective(metrics) -> float:
    metrics = copy.deepcopy(metrics)
    loss = metrics.pop("eval_loss", None)
    _ = metrics.pop("epoch", None)
    _ = metrics.pop("eval_runtime",None)
    _ = metrics.pop("eval_samples_per_second",None)
    return loss if len(metrics) == 0 else sum(metrics.values())

Starten der Hyperparametersuche:
- es werden 10 Versuche durchgeführt
- `direction="maximize"` sorgt dafür, dass das beste Modell anhand der Maximalen Punktzahl der Evaluationsmetriken ausgewählt wird
- die Methode returned als Ergebnis die verwendeten Hyperparameter des besten Versuchs

In [33]:
best_run = trainer_hyper.hyperparameter_search(n_trials=10, direction="maximize", hp_space= my_hp_space, compute_objective = my_objective)

[I 2021-03-10 10:31:24,212] A new study created in memory with name: no-name-dcf8cb3e-4041-4a79-a83d-eea016a447b9
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

Epoch,Training Loss,Validation Loss,Accurracy,Precision,Recall,F1,Runtime,Samples Per Second
1,No log,0.427662,0.862245,0.431122,0.500000,0.463014,1.528900,256.401000
2,No log,0.411555,0.862245,0.431122,0.500000,0.463014,1.554900,252.101000


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2021-03-10 10:31:57,839] Trial 0 finished with value: 2.2563810455689124 and parameters: {'learning_rate': 0.005184459747092712, 'num_train_epochs': 2, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 2.2563810455689124.
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.tr

Epoch,Training Loss,Validation Loss,Accurracy,Precision,Recall,F1,Runtime,Samples Per Second
1,No log,1.482403,0.150510,0.569767,0.507396,0.137026,1.535700,255.266000
2,No log,0.405821,0.862245,0.431122,0.500000,0.463014,1.595400,245.704000
3,No log,0.407504,0.862245,0.431122,0.500000,0.463014,1.669600,234.781000
4,No log,0.396716,0.862245,0.431122,0.500000,0.463014,1.648400,237.809000
5,No log,0.380799,0.862245,0.431122,0.500000,0.463014,1.534100,255.525000


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Pr

...

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2021-03-10 10:36:10,376] Trial 7 finished with value: 2.2563810455689124 and parameters: {'learning_rate': 2.7129478498085397e-06, 'num_train_epochs': 1, 'per_device_train_batch_size': 4}. Best is trial 2 with value: 2.627633618896888.
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertFor

Epoch,Training Loss,Validation Loss,Accurracy,Precision,Recall,F1,Runtime,Samples Per Second
1,No log,0.876797,0.137755,0.068878,0.500000,0.121076,1.481600,264.574000


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2021-03-10 10:36:16,300] Trial 8 pruned. 
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining mode

Epoch,Training Loss,Validation Loss,Accurracy,Precision,Recall,F1,Runtime,Samples Per Second
1,No log,0.534154,0.862245,0.431122,0.500000,0.463014,1.467600,267.101000


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2021-03-10 10:36:28,454] Trial 9 finished with value: 2.2563810455689124 and parameters: {'learning_rate': 0.002519008229003671, 'num_train_epochs': 1, 'per_device_train_batch_size': 64}. Best is trial 2 with value: 2.627633618896888.


Ausgabe des besten Versuchs:

In [34]:
best_run

BestRun(run_id='2', objective=2.627633618896888, hyperparameters={'learning_rate': 5.15697224769608e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 32})

Anschließend kann ein neuer Trainer erzeugt werden, welcher die ermittelten Hyperparameter des `best_run` nutzt.

In [35]:
#del trainer_hyper

In [36]:
#import torch

In [37]:
#torch.cuda.empty_cache()

Nachdem der `best_run` ermittelt wurde, kann nun ein neuer `Trainer` erzeugt werden:

In [38]:
trainer_hyper = Trainer(
    model_init=model_init,
    args=args,
    #Nutzt das gesamte Trainingsset
    train_dataset=dataset_preprocessed["train"],
    eval_dataset=dataset_preprocessed["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Die Hyperparamter des `best_run` werden mit folgendem Loop übertragen:

In [39]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer_hyper.args, n, v)

Ausgabe der `TrainingArguments` zur Kontrolle:

In [40]:
trainer_hyper.args

TrainingArguments(output_dir=task3_transformer_hyperparametersearch, overwrite_output_dir=False, do_train=False, do_eval=None, do_predict=False, evaluation_strategy=EvaluationStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=4, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5.15697224769608e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Mar10_10-31-14_fastai3, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=True, fp16_opt_level=O1, fp16_backend=auto, local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=False, dataloader_drop_last=False, eval_steps=500, dataloader_num_workers=0, past_index=-1, run_name=task3_transformer_hyperparametersearch, disable_tqdm=False, remove_unused_c

Anschließend kann das Modell mit den durch Optuna gefundenen Hyperparametern trainiert werden:

In [41]:
trainer_hyper.train()

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Epoch,Training Loss,Validation Loss,Accurracy,Precision,Recall,F1,Runtime,Samples Per Second
1,No log,0.288670,0.882653,0.835979,0.597414,0.629642,1.495600,262.098000
2,No log,0.319227,0.844388,0.693112,0.746384,0.713419,1.449900,270.370000
3,No log,0.443544,0.885204,0.767149,0.692253,0.720280,1.593500,246.003000
4,No log,0.537643,0.882653,0.765278,0.667434,0.699607,1.461300,268.250000
5,No log,0.576228,0.887755,0.782292,0.678172,0.712667,1.487100,263.593000


TrainOutput(global_step=245, training_loss=0.1504965334522481, metrics={'train_runtime': 78.8792, 'train_samples_per_second': 3.106, 'total_flos': 495417567043728, 'epoch': 5.0})

Evaluierung des Modells mit ungesehenen Testdaten:

In [42]:
trainer_hyper.evaluate(dataset_preprocessed["test"])

{'eval_loss': 0.5382338762283325,
 'eval_accurracy': 0.8612903225806452,
 'eval_precision': 0.7142946366723066,
 'eval_recall': 0.6769200480012001,
 'eval_f1': 0.6925257381868322,
 'eval_runtime': 3.5035,
 'eval_samples_per_second': 265.449,
 'epoch': 5.0}

Speichern des Models

In [43]:
trainer_hyper.save_model(f'./deepset_finetuned/offensive_language_deepset_{task}_hyperparametersearch_finetune_gem_germeval2019_split_makrof1')

In [44]:
tokenizer.save_pretrained(f'./deepset_finetuned/offensive_language_deepset_{task}_hyperparametersearch_finetune_gem_germeval2019_split_makrof1')

('./deepset_finetuned/offensive_language_deepset_task3_hyperparametersearch_finetune_gem_germeval2019_split_makrof1/tokenizer_config.json',
 './deepset_finetuned/offensive_language_deepset_task3_hyperparametersearch_finetune_gem_germeval2019_split_makrof1/special_tokens_map.json',
 './deepset_finetuned/offensive_language_deepset_task3_hyperparametersearch_finetune_gem_germeval2019_split_makrof1/vocab.txt',
 './deepset_finetuned/offensive_language_deepset_task3_hyperparametersearch_finetune_gem_germeval2019_split_makrof1/added_tokens.json')

#### Beispielhafte Anwendung des Modells
Um ein Transformers Modell einfach anzuwenden, nutzt man die Methode `pipeline`. Diese baut automatisch in Abhängigkeit vom übergebenen Task und Modell eine Pipeline. Die Pipeline kümmert sich um die nötigen Schritte um aus einer Stringeingabe eine Modellvorhersage zu erzeugen:

In [45]:
from transformers import pipeline
# der Task für SequenceClassification ist immer "sentiment-analysis"
classifier = pipeline("sentiment-analysis",model=f'./deepset_finetuned/offensive_language_deepset_{task}_hyperparametersearch_finetune_gem_germeval2019_split_makrof1')

In [46]:
classifier("Unglaublich wie beschissen du heute gespielt hast @ThomasMüller")

[{'label': 'EXPLICIT', 'score': 0.9987521171569824}]

In [47]:
classifier('Das klingt so als wäre da jemand als Baby vom Wickeltisch gefallen @UriGe')

[{'label': 'IMPLICIT', 'score': 0.6701172590255737}]